In [9]:
# Set root directory of the project as the current working directory
import os
initial_dir = os.getcwd()  # Save initial directory (notebooks/)
os.chdir('..')  # Move to project/

# Import Config from config.defaults and load_preprocessed_data, train_lstm_model, train_cnn_model from src.models
from config.defaults import Config
from src.models import load_preprocessed_data, load_saved_model

# Load the default configuration
config = Config()

import plotly.graph_objects as go
import numpy as np
import shap

In [2]:
# Load preprocessed data
model_task = "lstm_regression"

X_train, X_val, X_test, y_train_reg, y_val_reg, y_test_reg, metadata = load_preprocessed_data(model_task = model_task, eol_capacity=config.eol_capacity)

# Load the trained model
lstm_model = load_saved_model(model_task = model_task, config = config)

2025-03-07 09:45:27,472 - INFO - Loaded preprocessed data and metadata for lstm_regression with EOL 0.65
2025-03-07 09:45:27,731 - INFO - Loaded saved best LSTM model from experiments/models/lstm_regression_eol65_20250306_090920_best.keras


In [28]:
# Prediction function
f = lambda x: lstm_model.predict(x)

# Prepare background and baseline
background_2d = X_train.reshape(X_train.shape[0], -1)[:50]  # Shape: (50, 120)
data_2d = X_train.squeeze(axis=-1)  # Shape: (n_train, 120)
mean_per_timestep = np.mean(data_2d, axis=0)  # Shape: (120,)
baseline_sequence = mean_per_timestep.reshape(1, 120, 1)  # Shape: (1, 120, 1)

## SHAP

In [29]:
# Compute correlations for multiple sequences
n_sequences = 10  # Number of sequences to test
seq_indices = np.random.randint(0, len(X_test), size=n_sequences)
correlations = []

In [31]:
# Create a wrapper for the model's predict function
def predict_wrapper(X):
    # Reshape X from (n, 120) to (n, 120, 1)
    return lstm_model.predict(X.reshape(-1, 120, 1))

In [32]:
for idx in seq_indices:
    # Prepare test sequence
    test_3d = X_test[idx:idx+1]  # Shape: (1, 120, 1)
    test_2d = test_3d.reshape(1, -1)  # Shape: (1, 120)
    
    # SHAP Computation
    explainer = shap.KernelExplainer(predict_wrapper, background_2d)
    shap_values = explainer.shap_values(test_2d)
    shap_first = shap_values[0].flatten()  # Shape: (120,)
    
    # TimeSHAP Computation
    event_explanation = local_event_explainer(
        f,
        test_3d,
        baseline_sequence,
        pruned_idx=0,
        nsamples=10000
    )
    
    # Compute correlation
    correlation, _ = pearsonr(shap_first, event_explanation["Shapley Value"])
    correlations.append(correlation)
    print(f"Correlation for sequence {idx}: {correlation:.4f}")
    
    # Optional: Plot for each sequence
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(120), y=shap_first, mode='lines+markers', name='SHAP',
                             line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=event_explanation.index, y=event_explanation["Shapley Value"],
                             mode='lines+markers', name='TimeSHAP',
                             line=dict(color='red')))
    fig.update_layout(
        title=f"SHAP vs TimeSHAP for Test Sequence {idx}",
        xaxis_title="Timestep/Event Number",
        yaxis_title="Explanation Value",
        template="plotly_white"
    )
    fig.add_hline(y=0, line_dash="dash", line_color="gray")
    fig.show()

# Summarize correlations
print(f"\nMean Correlation: {np.mean(correlations):.4f}")
print(f"Median Correlation: {np.median(correlations):.4f}")
print(f"Min Correlation: {np.min(correlations):.4f}")
print(f"Max Correlation: {np.max(correlations):.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


  0%|          | 0/1 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


2025-03-07 10:55:41,326 - INFO - num_full_subsets = 1
2025-03-07 10:55:41,327 - INFO - remaining_weight_vector = [0.11638774 0.07825501 0.05919721 0.04776958 0.04015718 0.03472504
 0.0306557  0.027495   0.02497046 0.02290868 0.02119406 0.01974659
 0.0185091  0.01743969 0.01650692 0.01568675 0.01496052 0.01431345
 0.01373375 0.01321188 0.01274003 0.01231175 0.01192166 0.01156527
 0.01123875 0.01093887 0.01066285 0.0104083  0.01017315 0.0099556
 0.00975409 0.00956723 0.00939381 0.00923278 0.00908317 0.00894416
 0.00881499 0.008695   0.0085836  0.00848024 0.00838447 0.00829583
 0.00821397 0.00813852 0.00806919 0.00800569 0.00794777 0.00789523
 0.00784786 0.00780549 0.00776796 0.00773515 0.00770693 0.00768322
 0.00766393 0.00764899 0.00763835 0.00763198 0.00762986]
2025-03-07 10:55:41,328 - INFO - num_paired_subset_sizes = 59
2025-03-07 10:55:41,368 - INFO - weight_left = 0.8118838194333013


3575/3575 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step


...

2025-03-07 10:58:25,925 - INFO - num_full_subsets = 1
2025-03-07 10:58:25,955 - INFO - remaining_weight_vector = [0.11638774 0.07825501 0.05919721 0.04776958 0.04015718 0.03472504
 0.0306557  0.027495   0.02497046 0.02290868 0.02119406 0.01974659
 0.0185091  0.01743969 0.01650692 0.01568675 0.01496052 0.01431345
 0.01373375 0.01321188 0.01274003 0.01231175 0.01192166 0.01156527
 0.01123875 0.01093887 0.01066285 0.0104083  0.01017315 0.0099556
 0.00975409 0.00956723 0.00939381 0.00923278 0.00908317 0.00894416
 0.00881499 0.008695   0.0085836  0.00848024 0.00838447 0.00829583
 0.00821397 0.00813852 0.00806919 0.00800569 0.00794777 0.00789523
 0.00784786 0.00780549 0.00776796 0.00773515 0.00770693 0.00768322
 0.00766393 0.00764899 0.00763835 0.00763198 0.00762986]
2025-03-07 10:58:25,962 - INFO - num_paired_subset_sizes = 59
2025-03-07 10:58:26,067 - INFO - weight_left = 0.8118838194333013


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


2025-03-07 10:58:27,351 - INFO - np.sum(w_aug) = 120.00000000000001
2025-03-07 10:58:27,351 - INFO - np.sum(self.kernelWeights) = 0.9999999999999999
2025-03-07 10:58:27,572 - INFO - phi = [-0.11217801 -0.09392918 -0.0758166  -0.05433818 -0.03794048 -0.01889836
 -0.00400327  0.01094827  0.01594972  0.02004861  0.01847769  0.00628787
  0.00857248  0.00765298  0.00730804  0.          0.         -0.00228182
 -0.00888257 -0.00312139  0.         -0.00234827  0.          0.
  0.00255808  0.          0.          0.          0.00608665  0.
  0.00730806 -0.00184299 -0.00576404  0.          0.00274179  0.
  0.          0.          0.          0.         -0.00214786  0.00235482
  0.          0.          0.          0.         -0.00223236  0.
  0.          0.          0.         -0.00295323  0.          0.00201248
  0.          0.          0.          0.          0.          0.
  0.          0.          0.         -0.00570993  0.          0.
 -0.00455896  0.          0.         -0.00205011  0.00228

Correlation for sequence 272: 0.9465



Mean Correlation: 0.7039
Median Correlation: 0.9478
Min Correlation: -0.5363
Max Correlation: 0.9687


In [25]:
# Create a wrapper for the model's predict function
def predict_wrapper(X):
    # Reshape X from (n, 120) to (n, 120, 1)
    return lstm_model.predict(X.reshape(-1, 120, 1))

# Initialize the KernelExplainer with the 2D background data and the wrapper function
explainer = shap.KernelExplainer(predict_wrapper, background)

# Compute the SHAP values using the 2D test set
shap_values = explainer.shap_values(test_2d)

# Flatten it to a 1D array (120,)
shap_first = shap_values[0].flatten()

# Create the x-axis as timestep indices (0 to 119)
timesteps = np.arange(len(shap_first))

# Create a Plotly scatter plot
fig = go.Figure(data=go.Scatter(x=timesteps, y=shap_first, mode='lines+markers'))
fig.update_layout(
    title="First SHAP Values over Timesteps",
    xaxis_title="Timestep Index",
    yaxis_title="SHAP Value"
)
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


  0%|          | 0/1 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


2025-03-07 10:47:23,415 - INFO - num_full_subsets = 1
2025-03-07 10:47:23,416 - INFO - remaining_weight_vector = [0.11638774 0.07825501 0.05919721 0.04776958 0.04015718 0.03472504
 0.0306557  0.027495   0.02497046 0.02290868 0.02119406 0.01974659
 0.0185091  0.01743969 0.01650692 0.01568675 0.01496052 0.01431345
 0.01373375 0.01321188 0.01274003 0.01231175 0.01192166 0.01156527
 0.01123875 0.01093887 0.01066285 0.0104083  0.01017315 0.0099556
 0.00975409 0.00956723 0.00939381 0.00923278 0.00908317 0.00894416
 0.00881499 0.008695   0.0085836  0.00848024 0.00838447 0.00829583
 0.00821397 0.00813852 0.00806919 0.00800569 0.00794777 0.00789523
 0.00784786 0.00780549 0.00776796 0.00773515 0.00770693 0.00768322
 0.00766393 0.00764899 0.00763835 0.00763198 0.00762986]
2025-03-07 10:47:23,416 - INFO - num_paired_subset_sizes = 59
2025-03-07 10:47:23,454 - INFO - weight_left = 0.8118838194333013


72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


2025-03-07 10:47:23,766 - INFO - np.sum(w_aug) = 120.00000000000001
2025-03-07 10:47:23,766 - INFO - np.sum(self.kernelWeights) = 1.0
2025-03-07 10:47:23,901 - INFO - phi = [ 0.02124378  0.00665764  0.          0.         -0.00623207  0.00355916
 -0.014806    0.00617817  0.          0.01398734  0.02122288  0.
  0.         -0.00244307 -0.00463124  0.00743662 -0.0044769  -0.00363848
 -0.01088095 -0.01175544 -0.00642707  0.00855512  0.00709385  0.01003191
  0.00329732 -0.01799739 -0.01441561 -0.0102144   0.          0.00773642
 -0.00684286 -0.013469    0.01205829 -0.01783688  0.          0.00817126
 -0.00909222  0.01218446 -0.00410924  0.00680776  0.01653948  0.
  0.00703704 -0.00568613  0.00611024  0.         -0.00506551  0.
 -0.02184613  0.00542645  0.00944289 -0.01520694 -0.02876686  0.
  0.00628743 -0.0050066  -0.00727373  0.02006135 -0.01594917  0.00309915
  0.          0.01238171 -0.00919289  0.01099103 -0.00915708  0.00999377
  0.00672705 -0.00916957  0.0117973   0.01921255  0.0070

## Timeshap

In [26]:
from src.TimeSHAP import local_event_explainer, plot_local_event_explanation

# Extract the same sequence from the test set 
test_3d = X_test[0:1]

f = lambda x: lstm_model.predict(x)

background_3d = background.reshape(1, 120, 1)


# Call the functions
pruned_idx = 0  # Start explanation from the first timestep
time_shap_df = local_event_explainer(f, test_3d, background_3d, pruned_idx, nsamples=10000)
time_shap_fig = plot_local_event_explanation(time_shap_df)
time_shap_fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


2025-03-07 10:47:25,765 - INFO - num_full_subsets = 1
2025-03-07 10:47:25,766 - INFO - remaining_weight_vector = [0.11638774 0.07825501 0.05919721 0.04776958 0.04015718 0.03472504
 0.0306557  0.027495   0.02497046 0.02290868 0.02119406 0.01974659
 0.0185091  0.01743969 0.01650692 0.01568675 0.01496052 0.01431345
 0.01373375 0.01321188 0.01274003 0.01231175 0.01192166 0.01156527
 0.01123875 0.01093887 0.01066285 0.0104083  0.01017315 0.0099556
 0.00975409 0.00956723 0.00939381 0.00923278 0.00908317 0.00894416
 0.00881499 0.008695   0.0085836  0.00848024 0.00838447 0.00829583
 0.00821397 0.00813852 0.00806919 0.00800569 0.00794777 0.00789523
 0.00784786 0.00780549 0.00776796 0.00773515 0.00770693 0.00768322
 0.00766393 0.00764899 0.00763835 0.00763198 0.00762986]
2025-03-07 10:47:25,766 - INFO - num_paired_subset_sizes = 59
2025-03-07 10:47:25,844 - INFO - weight_left = 0.8118838194333013


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


2025-03-07 10:47:27,275 - INFO - np.sum(w_aug) = 120.00000000000001
2025-03-07 10:47:27,276 - INFO - np.sum(self.kernelWeights) = 0.9999999999999999
2025-03-07 10:47:27,523 - INFO - phi = [ 0.18000211  0.26423353  0.27160698  0.21685941  0.13221483  0.04214997
 -0.03071926 -0.09880717 -0.14103815 -0.14901333 -0.15078289 -0.14030062
 -0.11304305 -0.08723795 -0.04128571 -0.00960564  0.02779587  0.03994905
  0.06305583  0.06548853  0.05637232  0.05602887  0.0489487   0.02634557
  0.00260853 -0.00488099 -0.02059058 -0.0391893  -0.03020033 -0.02659862
 -0.02733562 -0.02731917 -0.02086301 -0.00587976  0.00296182  0.01443573
  0.02105442  0.01272135  0.01164783  0.01466704 -0.00358944  0.
  0.00386721 -0.00307622 -0.01420357 -0.01141503 -0.00142938 -0.01457911
 -0.00980128 -0.00372918  0.          0.          0.00825336  0.00182186
  0.00559874  0.00320828  0.00417191  0.          0.00125852  0.00421099
  0.0022686  -0.00297451 -0.00184832 -0.00334528 -0.00151924 -0.00276709
  0.00567349  0.0

In [27]:
# Compute correlation for quantitative comparison
from scipy.stats import pearsonr
correlation, _ = pearsonr(shap_first, time_shap_df["Shapley Value"])
print("Correlation between SHAP and TimeSHAP:", correlation)

Correlation between SHAP and TimeSHAP: 0.9787056616053802
